# Search and find assets

0. [Get Access Token](#authentication)
1. [Search for facets related toLibrary](#find-facets)
2. [Get data sets with Library.name facet](#use-facets)
3. [Get data sets with free text query](#get-tables)
4. [Get the 'datesets' search indices](#get-index)
5. [Get list of suggestions for 'Name' facet within the datasets index](#get-suggestions)
6. [Search for tables with 'WARRANTY' in the name that have been updated since yesterday](#get-last-updated)
7. [Find all crawled tables that contain columns classified as Date](#get-crawled-tables)
8. [Search for reports related to "inventory"](#get-crawled-tables)

> Please make sure you run [Imports and Global Variables](#imports) before executing anything else in this notebook. Also ensure you are authenticated by running [Get Access Token](#authentication).


## **Imports and Global Variables** <a id='imports'></a>

> _**NOTE**_: Run this cell before any of the others

- Imports packages and sets variables that will be used throughout the notebook

- Update `sasserver` to your server url

- You can change other variables to fit your use case. This will be used through out the exmaple.

In [2]:
import requests
import json
from datetime import datetime, timedelta

sasserver = "https://your_server_url"
username = 'your_username'
password = 'your_password'

cas_library = 'Samples'
cas_table_name = 'WATER_CLUSTER'

## Get Access Token <a id='authentication'></a>

- This gets and sets the `access_token` that will be used for the current section

- Run this periodically/as needed because sometimes, the authentication may expire after a certain amount of time


In [ ]:
url = f"{sasserver}/SASLogon/oauth/token"

payload = {
    'grant_type': 'password',
    'username': username,
    'password': password
}

headers = {
    'Authorization': 'Basic c2FzLmVjOg=='
}

response = requests.post(url, headers=headers, data=payload,verify=False)

if response.status_code == 200:
    access_token = 'Bearer ' + response.json()["access_token"]
    duration = round(int(response.json()['expires_in']) / 60)
    valid_until = datetime.now() + timedelta(minutes=duration)
    print(f"access_token is: {access_token}")
    print(f"This token will be valid until: {valid_until}")
else:
    print('Authentication failed')

## Search for facets related to Library <a id='find-facets'></a>

In [ ]:
url = f"{sasserver}/catalog/search/facets?q=Library"
headers = {
  'Authorization': access_token
}

response = requests.get(url, headers=headers, data=payload, verify=False)
print(json.dumps(response.json(), indent=4))

## Get CAS library with facet Library.name <a id='use-facets'></a>

In [ ]:
url = f"{sasserver}/catalog/search?q=Library.name:'{cas_library}'&indices=datasets"
headers = {
  'Authorization': access_token
}

response = requests.get(url, headers=headers, data=None, verify=False)

responseBody = response.json()
print(json.dumps(responseBody, indent=4))
    

## Get data sets with fuzzy search <a id='get-tables'></a>

In [ ]:
url = f"{sasserver}/catalog/search?q={cas_table_name}&indices=datasets"
headers = {
  'Authorization': access_token
}

response = requests.get(url, headers=headers, data=None, verify=False)

responseBody = response.json()
print(json.dumps(responseBody, indent=4))

## Get the 'datesets' search indices <a id='get-index'></a>

In [ ]:
url = f"{sasserver}/catalog/search/indices?filter=eq(name,'datasets')"
headers = {
    'Authorization': access_token
}
response = requests.get(url, headers=headers, data=None, verify=False)
print(json.dumps(response.json(), indent=4))

## Get list of suggestions for 'Name' facet within the datasets index <a id='get-suggestions'></a>

In [ ]:
url = f"{sasserver}/catalog/search/suggestions?facet=Name&indices=datasets&q="
headers = {
    'Authorization': access_token
}
response = requests.get(url, headers=headers, data=None, verify=False)
print(json.dumps(response.json(), indent=4))
    

## Search for tables whose names contain 'WARRANTY' and were updated on the previous day<a id='get-last-updated'></a>

In [ ]:
yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
today = datetime.today().strftime("%Y-%m-%d")
url = f"{sasserver}/catalog/search?indices=datasets&q=DateModified:[{yesterday} TO {today}]+Name:WARRANTY"
print(url)
headers = {
    'Authorization': access_token
}
response = requests.get(url, headers=headers, data=None, verify=False)
print(json.dumps(response.json(), indent=4))
    

## Find all crawled tables that contain columns classified as Date <a id='get-crawled-tables'></a>

In [ ]:
url = f"{sasserver}/catalog/search?indices=datasets&q=Column.semanticType:'date'"
headers = {
    'Authorization': access_token
}
response = requests.get(url, headers=headers, data=None, verify=False)
print(json.dumps(response.json(), indent=4))
    

## Search for reports related to "inventory" <a id='get-crawled-tables'></a>

In [ ]:
url = f"{sasserver}/catalog/search?q=inventory&indices=reports"
headers = {
    'Authorization': access_token
}
response = requests.get(url, headers=headers, data=None, verify=False)
print(json.dumps(response.json(), indent=4))